In [1]:
def catRecognition():
    
    print('Please select the category of each sub-system according to the following list:')
    # Later we must ask the user if any of the substrates has a stoichiometry other than 1? if yes,
    # we must add TFs to the model
    print('\n')
    print('diffusion: A <----> A*')
    print('simple reaction (1): A <---> B')
    print('simple reaction (2): A+B+... <---> C+D+...')
    print('enzyme catalyzed reaction (1): A+E <---> E+B')
    print('enzyme catalyzed reaction (2): A+E <---> B <---> E+C')
    print('\n')
    category=[]

        
    category.append(input('Enter the category of the reactions : '))

    return category

In [2]:
def simpleReactionTwo(fileName,modelName,*connections):
    import BondGraphTools as bgt

    
    [modelNum,subName,q,qVal,K,KVal,r,rVal,R,RVal,T,TVal,inputs_q,inputs_qVal,inputs_K,inputs_KVal]=getCellTwo(fileName)
    [P,zero_junc_s,zero_junc_p,reaction,one_junc_s,one_junc_p,S]=bgCompReactionTwo(modelNum,KVal,RVal,TVal,inputs_KVal)
    [connectivity_bonds_column,connectivity_bonds_row]=CBR_reactionTwoSorting(zero_junc_s,one_junc_s,reaction,one_junc_p,zero_junc_p,P)
    
    model=bgt.new(name=modelName)
    bgt.add(model,one_junc_s,reaction,one_junc_p)
    for m in range(modelNum):
        bgt.add(model,zero_junc_s[m],zero_junc_p[m],P[m])
    
    [Con_Mod,Port_Mod]=Index_C_Two(subName,*connections)
    connectivity_matrix_0=CM_CreationReactionTwo(modelNum,zero_junc_s,zero_junc_p,P)
    connectivity_matrix=Whole_CM_Two(Con_Mod,Port_Mod,zero_junc_s,zero_junc_p,connectivity_bonds_row,connectivity_matrix_0)   
    bonds=BondsConnectionTwo(connectivity_matrix,connectivity_bonds_row,connectivity_bonds_column)
    model=inputConnectionTwo(inputs_K,inputs_KVal,RVal,TVal,model,zero_junc_s)
    newDict=combineDictTwo(qVal,inputs_qVal)
    
    return [newDict,model,rVal]
    

[newDict,model,rVal]=simpleReactionTwo('Reactions.cellml','Reaction_Connection','reaction_1','out','reaction_2','in','reaction_2','out','reaction_3','in')

In [3]:
def getCellTwo(fileName):
    import requests
    import xml.etree.ElementTree as ET
    import pprint
    import numpy as np
    from copy import copy, deepcopy
    
    f = open(fileName,'r')
    text = f.read()
    root = ET.fromstring(text)
    components = root.findall('{http://www.cellml.org/cellml/1.1#}component')
    
    subName=[]
    for comp in components:
        subName.append(comp.attrib['name'])
    # Creating a list containing the names of reactions    
    for s in subName:
        if s=='input':
            inputIndex=deepcopy(subName.index(s))
            subName.pop(subName.index(s))
       
  #number of models in the file 
    modelNum=len(subName)
  #number of variables in each model 
    for comp in components:
        if comp.attrib['name']==subName[0]:
            compNum=len(comp) 
            
    
    variables=[]; els=[]; init=[]; inVars=[]; inputs=[]; inputVal=[]; Us=[];
    
    for comp in components:
        if comp.attrib['name'] != 'input':
            variables.append(comp.findall('{http://www.cellml.org/cellml/1.1#}variable'))
        else:
            inVars.append(comp.findall('{http://www.cellml.org/cellml/1.1#}variable'))
 
    for var in variables:
        for v in var:
            Us.append(v.attrib['units'])
            els.append(v.attrib['name'])
            if 'initial_value' in v.attrib: # if any initial value exists take it
                init.append(v.attrib['initial_value']) 
    inputs_unit=[]            
    for inp in inVars:
        for i in inp:
            inputs.append(i.attrib['name'])
            inputs_unit.append(i.attrib['units'])
            if 'initial_value' in i.attrib: # if any initial value exists take it
                inputVal.append(i.attrib['initial_value'])    
                
     # Separating the groups of elements, initial values & units in each model
    j=0
    elements=[]; init_vals=[]; units=[];
    for i in range(0,modelNum*compNum,compNum):    
        elements.append(els[i:i+compNum].copy())
        init_vals.append(init[i:i+compNum].copy())
        units.append(Us[i:i+compNum].copy())
        j+=1

    q={}; K={}; R={}; T={}; r={}; qVal={}; KVal={}; RVal={}; TVal={}; rVal={}; 
    for i in range(modelNum):
        q[i]=[]; qVal[i]=[]; K[i]=[]; R[i]=[]; T[i]=[]; r[i]=[];  KVal[i]=[]; RVal[i]=[]; TVal[i]=[]; rVal[i]=[];
        for j in range(compNum):
            if units[i][j] == "mol_per_m3":                
                q[i].append(elements[i][j])   
                qVal[i].append(init_vals[i][j]) 
            elif units[i][j] == "per_mol":
                K[i].append(elements[i][j])
                KVal[i].append(init_vals[i][j]) 
            elif units[i][j] == "mol_per_s":
                r[i].append(elements[i][j]) 
                rVal[i].append(init_vals[i][j])  
            elif units[i][j] == "J_per_mol_K":
                R[i].append(elements[i][j]) 
                RVal[i].append(init_vals[i][j]) 
            elif units[i][j] == "kelvin":
                T[i].append(elements[i][j]) 
                TVal[i].append(init_vals[i][j])  
                
    # Now we separate the "q" & "K" in the "input" module
    inputs_q=[]; inputs_K=[]; inputs_qVal=[]; inputs_KVal=[];
    for i in range(len(inputs)):
        if inputs_unit[i] == "mol_per_m3":
            inputs_q.append(inputs[i]) 
            inputs_qVal.append(inputVal[i])
        if inputs_unit[i] == "per_mol":
            inputs_K.append(inputs[i])  
            inputs_KVal.append(inputVal[i])  
        
        
    return [modelNum,subName,q,qVal,K,KVal,r,rVal,R,RVal,T,TVal,inputs_q,inputs_qVal,inputs_K,inputs_KVal]*1

[modelNum,subName,q,qVal,K,KVal,r,rVal,R,RVal,T,TVal,inputs_q,inputs_qVal,inputs_K,inputs_KVal]=getCellTwo('Reactions.cellml')

In [4]:
def bgCompReactionTwo(modelNum,KVal,RVal,TVal,inputs_KVal):
    import numpy as np
    import BondGraphTools as bgt
    
    # Here we create the required number of BG products, junctions & reactions

    P={};  zero_junc_s={}; zero_junc_p={}; reaction=[]; one_junc_s=[]; one_junc_p=[]; 
    for m in range(modelNum):
        P[m]=[]; zero_junc_s[m]=[]; zero_junc_p[m]=[];
        for v in range(len(KVal.values())): # the number of products can be gained from the number of K.values()
            P[m].append(bgt.new("Ce", library="BioChem", value={'k':float(KVal[m][v]), 'R':float(RVal[m][0]), 'T':float(TVal[m][0])}, name='P'+str(v)+'_'+str(m))) # Creating the required number of products in the name of P
            zero_junc_s[m].append(bgt.new("0", name='zero_junc_s_'+str(v)+'_'+str(m)))
            zero_junc_p[m].append(bgt.new("0", name='zero_junc_p_'+str(v)+'_'+str(m)))

        reaction.append(bgt.new("Re", library="BioChem", value={'r':None, 'R':float(RVal[m][0]), 'T':float(TVal[m][0])}, name='Reaction_'+str(m)))
            
        one_junc_s.append(bgt.new("1", name='one_junc_s_'+str(m)))
        one_junc_p.append(bgt.new("1", name='one_junc_p_'+str(m)))
    
  
    # Here we create the required number of BG inputs
    S=[];
    for v in range(len(inputs_KVal)):
        S.append(bgt.new("Ce", library="BioChem", value={'k':inputs_KVal[v], 'R':RVal[0], 'T':TVal[0]}, name='S'+str(v)))

    return [P,zero_junc_s,zero_junc_p,reaction,one_junc_s,one_junc_p,S]

[P,zero_junc_s,zero_junc_p,reaction,one_junc_s,one_junc_p,S]=bgCompReactionTwo(modelNum,KVal,RVal,TVal,inputs_KVal)

In [5]:
def CBR_reactionTwoSorting(zero_junc_s,one_junc_s,reaction,one_junc_p,zero_junc_p,P): # Since the number of inputs/outputs might be different 
 
    CBR=[]
    CBR.append([one_junc_s,reaction,one_junc_p])
    connectivity_bonds_row=[]
    for j in range(len(CBR[0][1])):
        for i in range(len(CBR[0])):
            connectivity_bonds_row.append(CBR[0][i][j])
    # Adding all the zero_junc_s components to the connectivity_bonds_row       
    for i in range(len(reaction)):
        for j in range(len(zero_junc_s[0])):
            connectivity_bonds_row.insert(i*(len(reaction)+len(zero_junc_s[i]))+j,zero_junc_s[i][j])
            
    # Adding all the zero_junc_p components to the connectivity_bonds_row
    for i in range(len(reaction)):
        for j in range(len(zero_junc_p[0])):
            connectivity_bonds_row.insert((i+1)*(len(reaction)+len(zero_junc_s[i]))+i*len(zero_junc_p[i]),zero_junc_p[i][len(zero_junc_p[i])-j-1])
    # Adding all the Product(P) components to the connectivity_bonds_row
    for i in range(len(reaction)):
        for j in range(len(P[0])):
            connectivity_bonds_row.insert((i+1)*(len(reaction)+len(zero_junc_s[i])+len(zero_junc_p[i]))+i*len(P[i]),P[i][len(zero_junc_p[i])-j-1])
    
    
            
    connectivity_bonds_column=connectivity_bonds_row
    
    return [connectivity_bonds_column,connectivity_bonds_row]

[connectivity_bonds_column,connectivity_bonds_row]=CBR_reactionTwoSorting(zero_junc_s,one_junc_s,reaction,one_junc_p,zero_junc_p,P)

import BondGraphTools as bgt
model=bgt.new(name='MY MODEL')
bgt.add(model,one_junc_s,reaction,one_junc_p)
for m in range(modelNum):
    bgt.add(model,zero_junc_s[m],zero_junc_p[m],P[m])

In [6]:
def Index_C_Two(subName,*connections):    
    
    connection=list(connections)
    conIter=int(len(connection)/4)
    connecting_models=[]
    ports=[]
    ConMod=[]
   # Dividing the elements in the input connections into lists with 4 elements, so each list represents ...
   # a separate connection between the modules
    for p in range(0,len(connection),4):
        ConMod.append(connection[p:p+4])
    
    # For the given inputs in connections, we assign numbers so the calculations can go on ...
    # For instance we assign {0,1} for {'in','out'} respectively and we assign {0,1,2,...} for the model components
    for m in range(conIter):
        for i in range(0,4):
            for j in range(len(subName)):
                if ConMod[m][i]==subName[j]:
                    connecting_models.append(j)
                    if ConMod[m][i+1]=='in':
                        ports.append(0)
                    if ConMod[m][i+1]=='out':
                        ports.append(1)

    
    
    # Now we split the connection modules and ports two by two to specify the detailed connections:
    Con_Mod=[]
    Port_Mod=[]
    for p in range(0,len(connecting_models),2):
        Con_Mod.append(connecting_models[p:p+2])
        Port_Mod.append(ports[p:p+2])
    
    return [Con_Mod,Port_Mod]

[Con_Mod,Port_Mod]=Index_C_Two(subName,'reaction_1','out','reaction_2','in','reaction_2','out','reaction_3','in')

In [7]:
def CM_CreationReactionTwo(modelNum,zero_junc_s,zero_junc_p,P):
    
    import numpy as np
    from copy import copy, deepcopy
    
    #  connectivity matrix for one sub-system
    lengthCM=len(zero_junc_s[0])+1+1+1+len(zero_junc_p[0])+len(P[0])
    connectivity_matrix_D=np.zeros(shape=(lengthCM,lengthCM))
    
    #inserting "1" for the connections ==> zero_junc_s , one_junc_s
    for s in range(len(zero_junc_s[0])):
        connectivity_matrix_D[s][len(zero_junc_s[0])]=1
    # insering "1" for the connection ==> one_junc_s , reaction
    connectivity_matrix_D[len(zero_junc_s[0])][len(zero_junc_s[0])+1]=1
    
    # insering "1" for the connection ==> reaction , one_junc_p
    connectivity_matrix_D[len(zero_junc_s[0])+1][len(zero_junc_s[0])+2]=1
    
    # insering "1" for the connection ==> one_junc_p , zero_junc_p
    for s in range(len(zero_junc_p[0])):
        connectivity_matrix_D[len(zero_junc_s[0])+2][len(zero_junc_s[0])+3+s]=1
    
    # insering "1" for the connection ==> zero_junc_p , P
    for s in range(len(zero_junc_p[0])):
        connectivity_matrix_D[len(zero_junc_s[0])+3+s][len(zero_junc_s[0])+3+len(zero_junc_p[0])+s]=1
    
    CM={}
    
    # Creating a dictionary with empty lists for adding the connectivity matrices for each sub-system
    for n in range(modelNum):
        CM[str(n)]=deepcopy(connectivity_matrix_D)
    

    Length=0
    for n in range(modelNum):
        Length+=len(CM[str(n)])
    connectivity_matrix_0=np.zeros((Length,Length))  

    
    k=0
    for n in range(modelNum):
        for i in range(k,k+len(CM[str(n)])):
            for j in range(k,k+len(CM[str(n)])):
                connectivity_matrix_0[i][j]=deepcopy(CM[str(n)][i-k][j-k])
        k+=len(CM[str(n)])
        
    return connectivity_matrix_0
    

connectivity_matrix_0=CM_CreationReactionTwo(modelNum,zero_junc_s,zero_junc_p,P)

In [8]:
def Whole_CM_Two(Con_Mod,Port_Mod,zero_junc_s,zero_junc_p,connectivity_bonds_row,connectivity_matrix_0):
# We want to connect the created sub-systems together by inserting "1"s in the connectivity matrix
    from copy import copy, deepcopy
    import numpy as np
    Name=[]
    for i in range(len(Con_Mod)):
        for j in range(len(Con_Mod[0])):
            m=deepcopy(Con_Mod[i][j])
            p=deepcopy(Port_Mod[i][j])
            if p == 0:
                for v in range(len(zero_junc_s)):
                        Name.append('zero_junc_s_'+str(v)+'_'+str(m))
            elif p == 1:
                for v in range(len(zero_junc_p)):
                        Name.append('zero_junc_p_'+str(v)+'_'+str(m))
           
    Connection_between_models=[]
    for name in Name:
        for index in range(len(connectivity_bonds_row)):
            if name == connectivity_bonds_row[index].name:
                Connection_between_models.append(index)
                break
         
    connectivity_matrix=deepcopy(connectivity_matrix_0)
    for i in range(0,len(Connection_between_models),len(zero_junc_s[0])+len(zero_junc_p[0])): # 6 by 6
        for j in range(len(zero_junc_s[0])):
            
            II=Connection_between_models[i+j]
            JJ=Connection_between_models[i+j+len(zero_junc_s[0])]
            connectivity_matrix[II][JJ]=1
        
    return connectivity_matrix

connectivity_matrix=Whole_CM_Two(Con_Mod,Port_Mod,zero_junc_s,zero_junc_p,connectivity_bonds_row,connectivity_matrix_0)

In [9]:
def BondsConnectionTwo(connectivity_matrix,connectivity_bonds_row,connectivity_bonds_column):
    import numpy as np
    import BondGraphTools as bgt
# Creating the bonds between the components based on the elements of the connectivity matrix
    Length=len(connectivity_matrix)
    bonds=[]  
    for i in range(Length):
        for j in range(Length):
            if connectivity_matrix[i][j]==1:
                bonds.append((connectivity_bonds_row[i],connectivity_bonds_column[j]))
            else:
                m=0
    
    
    # connecting the components for a bond graph representation of the system
    for head, tail in bonds:
        bgt.connect(head,tail)
    
    return bonds

bonds=BondsConnectionTwo(connectivity_matrix,connectivity_bonds_row,connectivity_bonds_column)

In [10]:
def inputConnectionTwo(inputs_K,inputs_KVal,RVal,TVal,model,zero_junc_s):
    import numpy as np
    import BondGraphTools as bgt
    
    A=[]
    for i in range(len(inputs_K)):
        A.append(bgt.new("Ce",library="BioChem", value={'k':float(inputs_KVal[i]), 'R':float(RVal[0][0]), 'T':float(TVal[0][0])}, name='A_'+str(i)))
        
    bgt.add(model,A) 
    
    for i in range(len(A)):
        bgt.connect(A[i],zero_junc_s[0][i])
    
    return model

model=inputConnectionTwo(inputs_K,inputs_KVal,RVal,TVal,model,zero_junc_s)

In [11]:
# Combining the initial values of the state variables
def combineDictTwo(qVal,inputs_qVal):
    incKey = 0
    newDict = {}
    for i in range(len(qVal)):
        for j in range(len(qVal[0])):
            newDict['x_'+str(incKey)] = qVal[i][j] 
            incKey+=1
    for i in inputs_qVal:
        newDict['x_'+str(incKey)] = i
        incKey+=1
        
    return newDict
